#Data Preprocessing

In [3]:
import pandas as pd
import numpy as np

In [4]:
train_data = pd.read_csv("./dataset/train_data.csv",encoding = 'CP949')
test_data = pd.read_csv("./dataset/test_data.csv",encoding = 'CP949')

In [7]:
train_data.head()

,발생년,발생년월일시,발생분,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,...,도로형태_대분류,도로형태,당사자종별_1당_대분류,당사자종별_1당,당사자종별_2당_대분류,당사자종별_2당,발생위치X_UTMK,발생위치Y_UTMK,경도,위도
0,2017,2017063023,14,야간,금,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,보행자,보행자,967633,1937456,127.134130,37.435704
1,2017,2017063021,45,야간,금,1,4,2,1,0,...,단일로,기타단일로,승용차,승용차,승용차,승용차,966456,1696405,127.131220,35.262534
2,2017,2017063021,50,야간,금,1,1,0,0,0,...,단일로,기타단일로,승용차,승용차,보행자,보행자,900057,1865697,126.379922,36.784142
3,2017,2017063019,10,야간,금,1,1,0,0,0,...,교차로,교차로부근,승용차,승용차,이륜차,이륜차,1095230,1764891,128.554945,35.875973
4,2017,2017063017,2,주간,금,1,2,1,0,0,...,단일로,교량위,이륜차,이륜차,화물차,화물차,947576,1946355,126.906786,37.514993


In [12]:
test_data.isnull().sum()

주야               5
요일               5
사망자수             9
사상자수            17
중상자수            14
경상자수             8
부상신고자수           2
발생지시도            5
발생지시군구          10
사고유형_대분류        12
사고유형_중분류        15
법규위반            13
도로형태_대분류        13
도로형태            13
당사자종별_1당_대분류     9
당사자종별_2당_대분류     7
dtype: int64

In [10]:
train_data.isnull().sum()

발생년             0
발생년월일시          0
발생분             0
주야              0
요일              0
사망자수            0
사상자수            0
중상자수            0
경상자수            0
부상신고자수          0
발생지시도           0
발생지시군구          0
사고유형_대분류        0
사고유형_중분류        0
사고유형            0
법규위반_대분류        0
법규위반            0
도로형태_대분류        0
도로형태            0
당사자종별_1당_대분류    0
당사자종별_1당        0
당사자종별_2당_대분류    0
당사자종별_2당        0
발생위치X_UTMK      0
발생위치Y_UTMK      0
경도              0
위도              0
dtype: int64

In [16]:
## 오브젝트 레이블 갯수확인 (null값 빼고)- 모델 설계할 때 필요.
col_list = train_data.columns
obj_col = []
for col in col_list :
    if train_data[col].dtypes =="object":
        obj_col.append(col)
        

In [17]:
obj_col

['주야',
 '요일',
 '발생지시도',
 '발생지시군구',
 '사고유형_대분류',
 '사고유형_중분류',
 '사고유형',
 '법규위반_대분류',
 '법규위반',
 '도로형태_대분류',
 '도로형태',
 '당사자종별_1당_대분류',
 '당사자종별_1당',
 '당사자종별_2당_대분류',
 '당사자종별_2당']

In [3]:
# testdata에 있는 columns 만 쓰기 위함
columns = test_data.columns

In [4]:
train_data = train_data[columns]

In [5]:
train_data.to_csv("./dataset/raw_train.csv", encoding="CP949", index = False)

In [6]:
# labelencoder를 위해 합침
merge = pd.concat([train_data, test_data])

merge.to_csv("./dataset/raw_merge.csv", encoding="CP949", index=False)

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
# 카테고리컬데이터은 레이블인코더로 바뀌고 테스트에 없는건 0으로 바뀜,
# 이를 위해 float데이터도 모두 1을 더해주고 없는 값만 0으로 해줌
# merge후 저장

objcol = [] # 레이블 갯수 확인용

for col in columns:
    if merge[col][0].dtype == object:
        le=LabelEncoder()
        y=merge[col].tolist()
        le.fit(y)
        merge[col]=le.transform(y)
        objcol.append(col)
    else:
        # float 데이터 처리
        y = merge[col].tolist()
        for i in range(len(y)):
            
            # nan이면 0으로
            if np.isnan(y[i]):
                y[i] = 0
            # 데이터 있으면 +1
            else:
                y[i] += 1
        merge[col] = y
        
merge.to_csv("./dataset/labeled_merge.csv", encoding="CP949", index = False)

In [9]:
## 오브젝트 레이블 갯수확인 (null값 빼고)- 모델 설계할 때 필요.
for i, obj in enumerate(objcol) :
    print(obj, ":", max(merge[obj].tolist()))

주야 : 2
요일 : 7
발생지시도 : 17
발생지시군구 : 208
사고유형_대분류 : 4
사고유형_중분류 : 19
법규위반 : 20
도로형태_대분류 : 9
도로형태 : 16
당사자종별_1당_대분류 : 12
당사자종별_2당_대분류 : 14


In [10]:
# 다시 분리 후 저장

train_data = merge[:train_data.shape[0]]
test_data = merge[train_data.shape[0]:]

train_data.to_csv("./dataset/labeled_train.csv", encoding="CP949", index = False)
test_data.to_csv("./dataset/labeled_test.csv", encoding="CP949", index = False)

#Build Model

In [11]:
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import to_categorical

Using TensorFlow backend.


In [12]:
## feature 갯수 지정
inputDim = test_data.shape[1]

## 학습을 위해 test셋의 빈칸 종류들을 넣기위함
## 전부 다 차있을 때 를 학습 시키기 위해 모두 1인 값을 넣어줌
valid_list = [[1]*inputDim]

## predict를 위해 만들어줌
test_masks = []

## 비어있는 곳은 0 차있는 곳은 1로 valid mask를 만들어서 넣어줌.

for i in range(test_data.shape[0]) :
    valid_mask = test_data.ix[i].tolist()
    
    for i, mask in enumerate(valid_mask):
        if not mask == 0 :
            valid_mask[i] = 1
    
    test_masks.append(valid_mask)
    
    if not valid_mask in valid_list :
        valid_list.append(valid_mask)
        
print("모두 1인 valid mask를 포함한 test data의 빈칸 뚤린 종류의 갯수:", len(valid_list))

모두 1인 valid mask를 포함한 test data의 빈칸 뚤린 종류의 갯수: 16


C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [13]:
data_input = Input(shape=(inputDim,), name="data_input")
valid_input = Input(shape=(inputDim,), name="valid_input")

## data_input이랑 valid_input 합치기
x = concatenate([data_input,valid_input])

## AutoEncoder 랑 비슷하게 만듬
x = Dense(64,activation='tanh')(x)
x = Dense(128,activation='tanh')(x)
x = Dense(64,activation='tanh')(x)
x = Dense(32,activation='tanh')(x)

## 카테고리컬 아웃풋은 레이블 갯수만큼 맞춰줌 활성화함수도 맞춰서 계산
output_1 = Dense(2, activation = 'softmax' , name = 'output_1')(x)
output_2 = Dense(7, activation = 'softmax', name = 'output_2')(x)
output_3 = Dense(1, activation = 'linear' , name = 'output_3')(x)
output_4 = Dense(1, activation = 'linear' , name = 'output_4')(x)
output_5 = Dense(1, activation = 'linear' , name = 'output_5')(x)
output_6 = Dense(1, activation = 'linear' , name = 'output_6')(x)
output_7 = Dense(1, activation = 'linear' , name = 'output_7')(x)
output_8 = Dense(17, activation = 'softmax', name = 'output_8')(x)
output_9 = Dense(208, activation = 'softmax', name = 'output_9')(x)
output_10 = Dense(4, activation = 'softmax', name = 'output_10')(x)
output_11 = Dense(19, activation = 'softmax', name = 'output_11')(x)
output_12 = Dense(20, activation = 'softmax', name = 'output_12')(x)
output_13 = Dense(9, activation = 'softmax', name = 'output_13')(x)
output_14 = Dense(16, activation = 'softmax', name = 'output_14')(x)
output_15 = Dense(12, activation = 'softmax', name = 'output_15')(x)
output_16 = Dense(14, activation = 'softmax', name = 'output_16')(x)

In [14]:
## model 설정

model = Model(inputs=[data_input, valid_input],
              outputs=[output_1, output_2, output_3, output_4, output_5, output_6, output_7, output_8, 
                       output_9, output_10, output_11, output_12, output_13, output_14, output_15, output_16])

model.compile(optimizer='Adam',
              loss = {'output_1': 'binary_crossentropy',
                     'output_2' : 'categorical_crossentropy',
                     'output_3' : 'mean_squared_error', 
                     'output_4' : 'mean_squared_error', 
                     'output_5' : 'mean_squared_error', 
                     'output_6' : 'mean_squared_error', 
                     'output_7' : 'mean_squared_error', 
                     'output_8' : 'categorical_crossentropy', 
                     'output_9' : 'categorical_crossentropy', 
                     'output_10' : 'categorical_crossentropy', 
                     'output_11' : 'categorical_crossentropy', 
                     'output_12' : 'categorical_crossentropy', 
                     'output_13' : 'categorical_crossentropy', 
                     'output_14' : 'categorical_crossentropy', 
                     'output_15' : 'categorical_crossentropy',
                     'output_16' : 'categorical_crossentropy'})

#Make Validation Set

In [15]:
def make_val(train_data, val_size):

    val_idx = np.random.choice(range(len(train_data)), val_size, replace=False)
    val_data = train_data[val_idx]
    sub_train_data = np.delete(train_data, val_idx, 0)

    return sub_train_data, val_data

#Train

In [16]:
# training 위해서 array로 바꿔줌

train_data = train_data[train_data.columns[:]]
train_data = np.array(train_data)
test_data = test_data[test_data.columns[:]]
test_data = np.array(test_data)

valid_list = np.array(valid_list)

In [17]:
## train_data 다시 보정, 0값은 null값이므로 기존의 0 레이블을 없애줌

train_data -= [1]

In [18]:

## 원하는 epochs 입력
epochs = 10
sub_epochs = 3

for i in range(epochs) :
    print("epochs {}/{}  \n".format(i+1,epochs))
        
    for j, valid in enumerate(valid_list) :
        print("{}/{}. valid_mask:".format(j+1, len(valid_list)),valid,"Start!\n")

        for k in range(sub_epochs) :

            ## validation set 만듦

            sub_train_data, val_data = make_val(train_data, (len(train_data)*3)//10)

            ##mask 씌우는 작업
            masked_sub_train_data = sub_train_data * valid
            masked_val_data = val_data * valid


            ## 카테고리컬 함수만 one-hot encoding으로 변환

            train_cate_1 = to_categorical(sub_train_data[:,0], num_classes=max(train_data[:,0]).astype(int)+1)
            train_cate_2 = to_categorical(sub_train_data[:,1], num_classes=max(train_data[:,1]).astype(int)+1)
            train_cate_3 = to_categorical(sub_train_data[:,7], num_classes=max(train_data[:,7]).astype(int)+1)
            train_cate_4 = to_categorical(sub_train_data[:,8], num_classes=max(train_data[:,8]).astype(int)+1)
            train_cate_5 = to_categorical(sub_train_data[:,9], num_classes=max(train_data[:,9]).astype(int)+1)
            train_cate_6 = to_categorical(sub_train_data[:,10], num_classes=max(train_data[:,10]).astype(int)+1)
            train_cate_7 = to_categorical(sub_train_data[:,11], num_classes=max(train_data[:,11]).astype(int)+1)
            train_cate_8 = to_categorical(sub_train_data[:,12], num_classes=max(train_data[:,12]).astype(int)+1)
            train_cate_9 = to_categorical(sub_train_data[:,13], num_classes=max(train_data[:,13]).astype(int)+1)
            train_cate_10 = to_categorical(sub_train_data[:,14], num_classes=max(train_data[:,14]).astype(int)+1)
            train_cate_11 = to_categorical(sub_train_data[:,15], num_classes=max(train_data[:,15]).astype(int)+1)

            val_cate_1 = to_categorical(val_data[:,0], num_classes=max(train_data[:,0]).astype(int)+1)
            val_cate_2 = to_categorical(val_data[:,1], num_classes=max(train_data[:,1]).astype(int)+1)
            val_cate_3 = to_categorical(val_data[:,7], num_classes=max(train_data[:,7]).astype(int)+1)
            val_cate_4 = to_categorical(val_data[:,8], num_classes=max(train_data[:,8]).astype(int)+1)
            val_cate_5 = to_categorical(val_data[:,9], num_classes=max(train_data[:,9]).astype(int)+1)
            val_cate_6 = to_categorical(val_data[:,10], num_classes=max(train_data[:,10]).astype(int)+1)
            val_cate_7 = to_categorical(val_data[:,11], num_classes=max(train_data[:,11]).astype(int)+1)
            val_cate_8 = to_categorical(val_data[:,12], num_classes=max(train_data[:,12]).astype(int)+1)
            val_cate_9 = to_categorical(val_data[:,13], num_classes=max(train_data[:,13]).astype(int)+1)
            val_cate_10 = to_categorical(val_data[:,14], num_classes=max(train_data[:,14]).astype(int)+1)
            val_cate_11 = to_categorical(val_data[:,15], num_classes=max(train_data[:,15]).astype(int)+1)


            print("sub_epochs {}/{} Start!\n".format(k+1,sub_epochs))

            model.fit({'data_input' : masked_sub_train_data, 'valid_input' : np.array([valid]*len(masked_sub_train_data))},
                     {'output_1': train_cate_1, 
                     'output_2': train_cate_2, 
                     'output_3': sub_train_data[:,2], 
                     'output_4': sub_train_data[:,3], 
                     'output_5': sub_train_data[:,4], 
                     'output_6': sub_train_data[:,5], 
                     'output_7': sub_train_data[:,6], 
                     'output_8': train_cate_3, 
                     'output_9': train_cate_4, 
                     'output_10': train_cate_5, 
                     'output_11': train_cate_6, 
                     'output_12': train_cate_7, 
                     'output_13': train_cate_8, 
                     'output_14': train_cate_9, 
                     'output_15': train_cate_10,
                     'output_16': train_cate_11 },
                      batch_size=32)


            ## validation 평가

            val_result = model.predict({'data_input' : masked_val_data, 'valid_input' : np.array([valid]*len(masked_val_data))})

            numerical_score = 0
            categorical_score = 0

            ## 평가하기위해 결과 쪼갬 및 one hot decoding

            result1 = np.array([[np.argmax(res)] for res in val_result[0]])
            result2 = np.array([[np.argmax(res)] for res in val_result[1]])
            result3 = val_result[2]
            result4 = val_result[3]
            result5 = val_result[4]
            result6 = val_result[5]
            result7 = val_result[6]
            result8 = np.array([[np.argmax(res)] for res in val_result[7]])
            result9 = np.array([[np.argmax(res)] for res in val_result[8]])
            result10 = np.array([[np.argmax(res)] for res in val_result[9]])
            result11 = np.array([[np.argmax(res)] for res in val_result[10]])
            result12 = np.array([[np.argmax(res)] for res in val_result[11]])
            result13 = np.array([[np.argmax(res)] for res in val_result[12]])
            result14 = np.array([[np.argmax(res)] for res in val_result[13]])
            result15 = np.array([[np.argmax(res)] for res in val_result[14]])
            result16 = np.array([[np.argmax(res)] for res in val_result[15]])

            ## numerical scoring

            numerical_score += np.exp(0-np.mean(((result3 - val_data[:,2])/ 1)**2))
            numerical_score += np.exp(0-np.mean(((result4 - val_data[:,3])/ 1)**2))
            numerical_score += np.exp(0-np.mean(((result5 - val_data[:,4])/ 1)**2))
            numerical_score += np.exp(0-np.mean(((result6 - val_data[:,5])/ 1)**2))
            numerical_score += np.exp(0-np.mean(((result7 - val_data[:,6])/ 1)**2))


            ## categorical scoring

            categorical_score += np.mean((result1 == val_cate_1).astype(float))
            categorical_score += np.mean((result2 == val_cate_2).astype(float))
            categorical_score += np.mean((result8 == val_cate_3).astype(float))
            categorical_score += np.mean((result9 == val_cate_4).astype(float))
            categorical_score += np.mean((result10 == val_cate_5).astype(float))
            categorical_score += np.mean((result11 == val_cate_6).astype(float))
            categorical_score += np.mean((result12 == val_cate_7).astype(float))
            categorical_score += np.mean((result13 == val_cate_8).astype(float))
            categorical_score += np.mean((result14 == val_cate_9).astype(float))
            categorical_score += np.mean((result15 == val_cate_10).astype(float))
            categorical_score += np.mean((result16 == val_cate_11).astype(float))


            print('\n Validation Accuracy - Numerical: {} Categorical: {}, Total: {}\n'
                  .format(numerical_score/5, categorical_score/11, (numerical_score + categorical_score)/16))
        
    
print('Finished!')

epochs 1/10  

1/16. valid_mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] Start!

sub_epochs 1/3 Start!

Epoch 1/1
17526/17526 [==============================] - 13704s 782ms/step - loss: 22.0858 - output_1_loss: 0.6659 - output_2_loss: 1.9357 - output_3_loss: 0.0835 - output_4_loss: 1.9112 - output_5_loss: 0.6597 - output_6_loss: 1.0475 - output_7_loss: 0.1030 - output_8_loss: 2.5447 - output_9_loss: 5.1242 - output_10_loss: 0.6215 - output_11_loss: 1.7843 - output_12_loss: 1.1368 - output_13_loss: 0.5594 - output_14_loss: 1.0568 - output_15_loss: 1.4862 - output_16_loss: 1.3654 5:48:26 - loss: 26.5598 - output_1_loss: 0.6975 - output_2_loss: 2.0070 - output_3_loss: 0.1101 - output_4_loss: 2.6482 - output_5_loss: 0.8754 - output_6_loss: 0.8673 - output_7_loss: 0.0958 - output_8_loss: 2.6855 - output_9_loss: 5.2353 - output_10_loss: 1.0481 - output_11_loss: 2.3485 - output_12_loss: 1.4627 - output_13_loss: 1.1294 - output - ETA: 3:16:12 - loss: 24.6881 - output_1_loss: 0.6930 

17526/17526 [==============================] - 12s 684us/step - loss: 10.5963 - output_1_loss: 0.0257 - output_2_loss: 0.5642 - output_3_loss: 0.0515 - output_4_loss: 0.9196 - output_5_loss: 0.3561 - output_6_loss: 0.3819 - output_7_loss: 0.3600 - output_8_loss: 1.0929 - output_9_loss: 2.9378 - output_10_loss: 0.1376 - output_11_loss: 1.4586 - output_12_loss: 0.9543 - output_13_loss: 0.1134 - output_14_loss: 0.3923 - output_15_loss: 0.4192 - output_16_loss: 0.4310

 Validation Accuracy - Numerical: 0.42323040007217294 Categorical: 0.07295199902739288, Total: 0.18241399935388664

sub_epochs 3/3 Start!

Epoch 1/1
17526/17526 [==============================] - 11s 654us/step - loss: 9.6080 - output_1_loss: 0.0267 - output_2_loss: 0.5226 - output_3_loss: 0.0579 - output_4_loss: 0.5610 - output_5_loss: 0.3609 - output_6_loss: 0.3920 - output_7_loss: 0.1241 - output_8_loss: 1.0287 - output_9_loss: 2.8383 - output_10_loss: 0.1022 - output_11_loss: 1.4113 - output_12_loss: 0.9259 - output_13_l

17526/17526 [==============================] - 11s 651us/step - loss: 11.8155 - output_1_loss: 0.0206 - output_2_loss: 0.2283 - output_3_loss: 0.0569 - output_4_loss: 0.5565 - output_5_loss: 0.2341 - output_6_loss: 0.1944 - output_7_loss: 0.3763 - output_8_loss: 3.1487 - output_9_loss: 5.3552 - output_10_loss: 0.0080 - output_11_loss: 0.3298 - output_12_loss: 0.2407 - output_13_loss: 0.0922 - output_14_loss: 0.3514 - output_15_loss: 0.3062 - output_16_loss: 0.3163

 Validation Accuracy - Numerical: 0.42906422380795234 Categorical: 0.07824937550531962, Total: 0.18787901559989234

sub_epochs 2/3 Start!

Epoch 1/1
17526/17526 [==============================] - 11s 651us/step - loss: 10.0793 - output_1_loss: 0.0116 - output_2_loss: 0.1862 - output_3_loss: 0.0590 - output_4_loss: 0.1988 - output_5_loss: 0.2048 - output_6_loss: 0.1369 - output_7_loss: 0.1230 - output_8_loss: 2.6427 - output_9_loss: 5.0362 - output_10_loss: 0.0048 - output_11_loss: 0.3008 - output_12_loss: 0.1948 - output_13_

17526/17526 [==============================] - 12s 676us/step - loss: 7.1883 - output_1_loss: 0.0350 - output_2_loss: 0.2252 - output_3_loss: 0.0591 - output_4_loss: 1.7400 - output_5_loss: 0.1979 - output_6_loss: 1.1883 - output_7_loss: 0.3439 - output_8_loss: 0.3628 - output_9_loss: 1.6047 - output_10_loss: 0.0115 - output_11_loss: 0.2821 - output_12_loss: 0.2108 - output_13_loss: 0.0697 - output_14_loss: 0.3211 - output_15_loss: 0.2841 - output_16_loss: 0.2519

 Validation Accuracy - Numerical: 0.4259140663755231 Categorical: 0.07532874873729546, Total: 0.18488666049924157

3/16. valid_mask: [1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.] Start!

sub_epochs 1/3 Start!

Epoch 1/1
17526/17526 [==============================] - 12s 661us/step - loss: 6.2196 - output_1_loss: 0.0156 - output_2_loss: 0.2036 - output_3_loss: 0.0613 - output_4_loss: 1.3590 - output_5_loss: 0.8900 - output_6_loss: 0.2343 - output_7_loss: 0.1429 - output_8_loss: 0.3266 - output_9_loss: 1.5680 - output_10_lo

17526/17526 [==============================] - 12s 662us/step - loss: 5.0268 - output_1_loss: 0.0051 - output_2_loss: 0.0464 - output_3_loss: 0.0627 - output_4_loss: 0.2686 - output_5_loss: 0.1901 - output_6_loss: 0.1944 - output_7_loss: 0.1218 - output_8_loss: 0.1534 - output_9_loss: 2.9419 - output_10_loss: 0.0090 - output_11_loss: 0.1376 - output_12_loss: 0.1461 - output_13_loss: 0.0612 - output_14_loss: 0.3006 - output_15_loss: 0.2007 - output_16_loss: 0.1872

 Validation Accuracy - Numerical: 0.3512712463538672 Categorical: 0.07549114174405852, Total: 0.16167242443462373

11/16. valid_mask: [1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.] Start!

sub_epochs 1/3 Start!

Epoch 1/1
17526/17526 [==============================] - 12s 666us/step - loss: 6.3142 - output_1_loss: 0.0092 - output_2_loss: 0.0726 - output_3_loss: 0.0519 - output_4_loss: 0.7178 - output_5_loss: 0.5968 - output_6_loss: 0.6259 - output_7_loss: 0.0682 - output_8_loss: 0.1626 - output_9_loss: 2.9362 - output_10_l

...

17526/17526 [==============================] - 12s 678us/step - loss: 3.5638 - output_1_loss: 0.1941 - output_2_loss: 0.0130 - output_3_loss: 0.0559 - output_4_loss: 1.0732 - output_5_loss: 0.7593 - output_6_loss: 0.1913 - output_7_loss: 0.2656 - output_8_loss: 0.0308 - output_9_loss: 0.4691 - output_10_loss: 0.0103 - output_11_loss: 0.1016 - output_12_loss: 0.0814 - output_13_loss: 0.0461 - output_14_loss: 0.1065 - output_15_loss: 0.0701 - output_16_loss: 0.0954

 Validation Accuracy - Numerical: 0.4234494307829543 Categorical: 0.07484243346693589, Total: 0.18378212012819165

4/16. valid_mask: [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] Start!

sub_epochs 1/3 Start!

Epoch 1/1
17526/17526 [==============================] - 12s 672us/step - loss: 7.8859 - output_1_loss: 0.2224 - output_2_loss: 0.0323 - output_3_loss: 0.0589 - output_4_loss: 3.5233 - output_5_loss: 1.1252 - output_6_loss: 1.3936 - output_7_loss: 0.2709 - output_8_loss: 0.0481 - output_9_loss: 0.5539 - output_10_lo

17526/17526 [==============================] - 13s 723us/step - loss: 4.8959 - output_1_loss: 0.0015 - output_2_loss: 0.0089 - output_3_loss: 0.0589 - output_4_loss: 0.3799 - output_5_loss: 0.5375 - output_6_loss: 0.4265 - output_7_loss: 0.2735 - output_8_loss: 0.0206 - output_9_loss: 2.7340 - output_10_loss: 0.0040 - output_11_loss: 0.0893 - output_12_loss: 0.0717 - output_13_loss: 0.0374 - output_14_loss: 0.1012 - output_15_loss: 0.0932 - output_16_loss: 0.0575

 Validation Accuracy - Numerical: 0.41545744184740946 Categorical: 0.07533920264097635, Total: 0.18162615239298668

sub_epochs 3/3 Start!

Epoch 1/1
17526/17526 [==============================] - 13s 726us/step - loss: 4.5117 - output_1_loss: 0.0016 - output_2_loss: 0.0097 - output_3_loss: 0.0605 - output_4_loss: 0.2500 - output_5_loss: 0.3815 - output_6_loss: 0.3865 - output_7_loss: 0.3083 - output_8_loss: 0.0215 - output_9_loss: 2.7115 - output_10_loss: 0.0041 - output_11_loss: 0.0875 - output_12_loss: 0.0665 - output_13_lo

17526/17526 [==============================] - 13s 721us/step - loss: 3.9171 - output_1_loss: 0.6906 - output_2_loss: 0.0503 - output_3_loss: 0.0603 - output_4_loss: 0.3199 - output_5_loss: 0.0928 - output_6_loss: 0.1278 - output_7_loss: 0.3117 - output_8_loss: 0.0168 - output_9_loss: 0.4420 - output_10_loss: 0.0036 - output_11_loss: 0.1034 - output_12_loss: 0.0862 - output_13_loss: 0.0364 - output_14_loss: 0.1088 - output_15_loss: 1.4077 - output_16_loss: 0.0589

 Validation Accuracy - Numerical: 0.40887613618228436 Categorical: 0.07555484226433912, Total: 0.179717746613697

sub_epochs 3/3 Start!

Epoch 1/1
17526/17526 [==============================] - 13s 729us/step - loss: 3.5158 - output_1_loss: 0.6854 - output_2_loss: 0.0276 - output_3_loss: 0.0638 - output_4_loss: 0.1760 - output_5_loss: 0.1102 - output_6_loss: 0.1214 - output_7_loss: 0.1251 - output_8_loss: 0.0097 - output_9_loss: 0.4281 - output_10_loss: 0.0030 - output_11_loss: 0.1005 - output_12_loss: 0.0806 - output_13_loss

17526/17526 [==============================] - 13s 763us/step - loss: 4.2184 - output_1_loss: 0.0012 - output_2_loss: 0.0125 - output_3_loss: 0.0580 - output_4_loss: 0.2942 - output_5_loss: 0.1293 - output_6_loss: 0.1472 - output_7_loss: 0.3143 - output_8_loss: 0.0360 - output_9_loss: 0.4370 - output_10_loss: 0.0958 - output_11_loss: 1.3760 - output_12_loss: 0.9559 - output_13_loss: 0.0298 - output_14_loss: 0.0914 - output_15_loss: 0.0649 - output_16_loss: 0.1750

 Validation Accuracy - Numerical: 0.4151721799458666 Categorical: 0.075008380459127, Total: 0.1813095677987331

sub_epochs 3/3 Start!

Epoch 1/1
17526/17526 [==============================] - 13s 764us/step - loss: 4.0576 - output_1_loss: 0.0018 - output_2_loss: 0.0237 - output_3_loss: 0.0629 - output_4_loss: 0.3112 - output_5_loss: 0.1644 - output_6_loss: 0.1437 - output_7_loss: 0.2639 - output_8_loss: 0.0246 - output_9_loss: 0.4168 - output_10_loss: 0.0627 - output_11_loss: 1.3552 - output_12_loss: 0.9293 - output_13_loss: 

17526/17526 [==============================] - 14s 822us/step - loss: 7.5306 - output_1_loss: 3.5642e-04 - output_2_loss: 2.0830 - output_3_loss: 0.0621 - output_4_loss: 0.2263 - output_5_loss: 0.1309 - output_6_loss: 0.1328 - output_7_loss: 0.3277 - output_8_loss: 0.1299 - output_9_loss: 1.2282 - output_10_loss: 0.0103 - output_11_loss: 1.2948 - output_12_loss: 0.9605 - output_13_loss: 0.2074 - output_14_loss: 0.5642 - output_15_loss: 0.0542 - output_16_loss: 0.1180

 Validation Accuracy - Numerical: 0.43060366639774517 Categorical: 0.07749062911449102, Total: 0.18783845326550794

sub_epochs 3/3 Start!

Epoch 1/1
17526/17526 [==============================] - 14s 826us/step - loss: 6.7078 - output_1_loss: 2.3691e-04 - output_2_loss: 2.0796 - output_3_loss: 0.0598 - output_4_loss: 0.1167 - output_5_loss: 0.0786 - output_6_loss: 0.1171 - output_7_loss: 0.0877 - output_8_loss: 0.0362 - output_9_loss: 1.0654 - output_10_loss: 0.0073 - output_11_loss: 1.2892 - output_12_loss: 0.9473 - outp

In [36]:
model.save('keras_model.h5')

#Predict

In [28]:
result = model.predict({'data_input' : test_data, 'valid_input' : np.array(test_masks)})

#Make result file

In [29]:
## +[1] 은 label decoder를 위해 해줌

result1 = np.array([[np.argmax(res)] for res in result[0]]) + [1]
result2 = np.array([[np.argmax(res)] for res in result[1]]) + [1]
result3 = result[2]
result4 = result[3]
result5 = result[4]
result6 = result[5]
result7 = result[6]
result8 = np.array([[np.argmax(res)] for res in result[7]]) + [1]
result9 = np.array([[np.argmax(res)] for res in result[8]]) + [1]
result10 = np.array([[np.argmax(res)] for res in result[9]]) + [1]
result11 = np.array([[np.argmax(res)] for res in result[10]]) + [1]
result12 = np.array([[np.argmax(res)] for res in result[11]]) + [1]
result13 = np.array([[np.argmax(res)] for res in result[12]]) + [1]
result14 = np.array([[np.argmax(res)] for res in result[13]]) + [1]
result15 = np.array([[np.argmax(res)] for res in result[14]]) + [1]
result16 = np.array([[np.argmax(res)] for res in result[15]]) + [1]

In [30]:
## column들 다시 합쳐주기

result = np.hstack((result1, result2, result3, result4, result5, result6, result7, result8,
         result9, result10, result11, result12, result13, result14, result15, result16))

In [31]:
## 역으로 되돌려 주기 위함
merge = pd.read_csv("./dataset/raw_merge.csv", encoding='CP949')

decoded_result = []
for i, col in enumerate(columns):
    if type(merge[col][0]) == str:
        le=LabelEncoder()
        y=merge[col].tolist()
        le.fit(y)
        z = le.inverse_transform(result[:,i].astype(int))
        z = z.reshape(-1,1)
        ## decoded_result 에 아무것도 없으면 hstack 하는데 에러남
        if not len(decoded_result):
            decoded_result = z
        else:
            decoded_result = np.hstack((decoded_result,z))
        
    else:
        z = result[:,i]
        z = z.reshape(-1,1)
        decoded_result = np.hstack((decoded_result,z))

C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an 

In [32]:
## decoded_result file을 data frame으로 바꿔서 csv파일로 저장

decoded_result_df = {columns[0] : decoded_result[:,0], 
                     columns[1] : decoded_result[:,1], 
                     columns[2] : decoded_result[:,2], 
                     columns[3] : decoded_result[:,3], 
                     columns[4] : decoded_result[:,4], 
                     columns[5] : decoded_result[:,5], 
                     columns[6] : decoded_result[:,6], 
                     columns[7] : decoded_result[:,7], 
                     columns[8] : decoded_result[:,8], 
                     columns[9] : decoded_result[:,9], 
                     columns[10] : decoded_result[:,10], 
                     columns[11] : decoded_result[:,11], 
                     columns[12] : decoded_result[:,12], 
                     columns[13] : decoded_result[:,13], 
                     columns[14] : decoded_result[:,14], 
                     columns[15] : decoded_result[:,15], }

decoded_result_df = pd.DataFrame(decoded_result_df, columns = columns)

decoded_result_df.to_csv("./dataset/decoded_result.csv", encoding="CP949", index = False)

In [33]:
result_kor = pd.read_csv("./result_kor.csv",encoding = 'CP949')
result_kor_HH = pd.read_csv("./result_kor.csv",encoding = 'CP949')

In [34]:
for col in ["행","열"]:
    if type(result_kor[col][0]) == str:
        # 열 데이터 처리
        le=LabelEncoder()
        y=result_kor[col].tolist()
        le.fit(y)
        result_kor[col]=le.transform(y)
    else:
        # 행 데이터 처리
        y = result_kor[col].tolist()
        for i in range(len(y)):
            #list 위치에 대한 보정값
            y[i] -= 2
            
        result_kor[col] = y

In [35]:
for i in range(result_kor.shape[0]):
    result_kor_HH["값"][i] = decoded_result[result_kor["행"][i],result_kor["열"][i]]

C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\envs\myvenv\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
result_kor_HH.to_csv("./result_kor_HH.csv", encoding="CP949", index = False)

In [37]:
import keras
print(keras.__version__)

2.2.0


In [39]:
import tensorflow
print(tensorflow.__version__)

1.10.0
